In [1]:
import os, sys, time
sys.path.append(r'C:\Users\charl\Documents\GitHub\GOST_PublicGoods\GOSTNets\GOSTNets')
import importlib
import GOSTnet as gn
importlib.reload(gn)
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.wkt import loads
from shapely.geometry import Point, LineString, MultiLineString
import networkx as nx
import osmnx as ox

peartree version: 0.6.0 
networkx version: 2.2 
matplotlib version: 2.2.2 
osmnx version: 0.8.2 
peartree version: 0.6.0 
networkx version: 2.2 
matplotlib version: 2.2.2 
osmnx version: 0.8.2 


In [2]:
npath = r'C:\Users\charl\Documents\GOST\NetClean\output\YEM'
npickle = 'YEM_processed.pickle'
bpath = r'C:\Users\charl\Documents\GOST\Yemen'
opath = os.path.join(bpath, 'origins')
dpath = os.path.join(bpath, 'facility_files')
wpath = r'C:\Users\charl\Documents\GOST\Yemen\YEM'

### Loads saved graph from pickle

In [3]:
G = nx.read_gpickle(os.path.join(npath, npickle))

list_of_subgraphs = list(nx.strongly_connected_component_subgraphs(G))

G_2 = list_of_subgraphs[0]
list_of_small_subgraphs = list_of_subgraphs[1:]

node_bunch = []
edge_bunch = []

for subgraph in list_of_small_subgraphs:
    if subgraph.number_of_nodes() > 50 and subgraph.number_of_nodes() != G_2.number_of_nodes():
        for u, data in subgraph.nodes(data = True):
            node_bunch.append((u, data))

        for u, v, data in subgraph.edges(data = True):
            edge_bunch.append((u, v, data))
    
G_2.add_nodes_from(node_bunch)
G_2.add_edges_from(edge_bunch)

G = G_2
print(G.number_of_edges())
gn.save(G, 'G', wpath)

283769


In [4]:
G_salty = gn.salt_long_lines(G, 'epsg:4326', 'epsg:32638', thresh = 2000, factor = 0.001)
G_salty = nx.convert_node_labels_to_integers(G_salty)

print('check: salting process has left number of connected components unchanged')
print(len(list(nx.strongly_connected_component_subgraphs(G))),
      ' | ', 
      len(list(nx.strongly_connected_component_subgraphs(G_salty))))

Identified 5986 unique edge(s) longer than 2000. 
Beginning new node creation...
44268 new edges added and 11956 removed to bring total edges to 316081
16148 new nodes added to bring total nodes to 121595
check: salting process has left number of connected components unchanged
2  |  2


In [5]:
importlib.reload(gn)

print('adding traverse time edge property...')
# define speed limit dictionary
speed_dict = {
    'residential':25,
    'unclassified':15,
    'track':15,
    'tertiary':40,
    'secondary':50,
    'primary':60,
    'trunk':50,
    'service':15,
    'road':15,
    'trunk_link':50,
    'secondary_link':50,
    'primary_link':60,
    'tertiary_link':40}

# add traverse time property into 'time' edge attribute 
G_salty_time = gn.convert_network_to_time(G_salty, 
                                          distance_tag = 'length', 
                                          road_col = 'infra_type', 
                                          graph_type = 'drive', 
                                          speed_dict = speed_dict, 
                                          walk_speed = 4,
                                          factor = 1000
                                         )
gn.example_edge(G_salty_time, 2)

gn.save(G_salty_time, 'G_salty_time', wpath)

peartree version: 0.6.0 
networkx version: 2.2 
matplotlib version: 2.2.2 
osmnx version: 0.8.2 
adding traverse time edge property...
(0, 40443, {'Wkt': 'LINESTRING (44.0580736 13.5904793, 44.058066 13.5904398)', 'id': 29471, 'infra_type': 'track', 'osm_id': 117965664, 'country': 'YEM', 'key': 'edge_29471', 'length': 4.446831582994608, 'Type': 'legitimate', 'time': 1.0672395799187058, 'mode': 'drive'})
(0, 87370, {'Wkt': 'LINESTRING (44.0580736 13.5904793, 44.0581423 13.590818)', 'id': 29485, 'infra_type': 'track', 'osm_id': 117965673, 'country': 'YEM', 'key': 'edge_29485', 'length': 38.20274144564127, 'Type': 'legitimate', 'time': 9.168657946953905, 'mode': 'drive'})


### Manual Addition of Missing Roads

In [6]:
missed_edges = []

st_node = 102515
end_node = 111784
st_point = Point(G_salty_time.nodes()[st_node]['x'],G_salty_time.nodes()[st_node]['y'])
end_point = Point(G_salty_time.nodes()[end_node]['x'],G_salty_time.nodes()[end_node]['y'])
lin = LineString([st_point,end_point])
real_length = 115

data = {'Wkt':lin,
       'id':max(nx.get_edge_attributes(G_salty_time,'id').values())+1,
       'infra_type':'service',
       'country':'YEM',
       'key':'manual_edge_1',
       'length':real_length, 
       'Type':'manual_edge',
       'time':float((real_length / 15)),
       'mode':'drive'}
missed_edges.append((st_node, end_node, data))
missed_edges.append((end_node, st_node, data))

G_salty_time.add_edges_from(missed_edges)

G_salty = nx.convert_node_labels_to_integers(G_salty)

gn.save(G_salty_time, 'G_salty_time', wpath)

### Snap Destination Files to Network

In [7]:
dfiles = ['YEHNP hospitals.csv',
          'YEHNP PHCs_unicef.csv',
          'HeRAMs_2016v2018_damages and functionality_cleaned.csv',
          'all facilities_herams.csv']
wpath = dpath

for dfile in dfiles:
    # Read in
    dest_df = pd.read_csv(os.path.join(os.path.join(dpath, dfile)), encoding = "ISO-8859-1")
    
    # Ensure coordinates are floats
    dest_df.Longitude = dest_df.Longitude.astype(float)
    dest_df.Latitude = dest_df.Latitude.astype(float)
    
    # Drop entries with no coordinates    
    dest_df2 = dest_df.copy()
    dest_df2 = dest_df2.loc[(dest_df2.Longitude != 0)]
    dest_df2 = dest_df2.loc[(dest_df2.Longitude != None)]
    dest_df = dest_df2
    
    # Generate Geometries
    dest_df['geometry'] = list(zip(dest_df.Longitude, dest_df.Latitude))
    dest_df['geometry'] = dest_df['geometry'].apply(Point)
    dest_df = gpd.GeoDataFrame(dest_df, geometry = 'geometry', crs = {'init':'espg:4326'})
    
    # Perform snap
    time.ctime()
    start = time.time()
    df = gn.pandana_snap(G_salty_time, dest_df, 'epsg:4326','epsg:32638', add_dist_to_node_col = True)
    
    # Save to file
    df.to_csv(os.path.join(wpath, dfile.replace('.csv', '_snapped.csv')))

    print('time elapsed: %d seconds' % (time.time() - start))

C:\Users\charl\Documents\GitHub\GOST_PublicGoods\GOSTNets\GOSTNets\GOSTnet.py:1599: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  G_tree = spatial.KDTree(node_gdf[['x','y']].as_matrix())
C:\Users\charl\Documents\GitHub\GOST_PublicGoods\GOSTNets\GOSTNets\GOSTnet.py:1601: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  distances, indices = G_tree.query(in_df[['x','y']].as_matrix())


time elapsed: 9 seconds
time elapsed: 10 seconds
time elapsed: 10 seconds
time elapsed: 37 seconds


### Snap Origin Points to Networks

In [8]:
ofile = r'origins_1km.csv'
ofiles = [ofile]
wpath = opath 

for ofile in ofiles:
    
    # Read in
    dest_df = pd.read_csv(os.path.join(os.path.join(opath, ofile)), encoding = "ISO-8859-1")
    dest_df['geometry'] = dest_df['geometry'].apply(loads)
    dest_df = gpd.GeoDataFrame(dest_df, geometry = 'geometry', crs = {'init':'espg:4326'})
    
    # Perform snap
    print('Beginning snap')
    
    time.ctime()
    start = time.time()
    df = gn.pandana_snap(G_salty_time, dest_df, 'epsg:4326','epsg:32638', add_dist_to_node_col = True)
    
    # Save to file
    df.to_csv(os.path.join(wpath, ofile.replace('.csv', '_snapped.csv')))
    compath = r'C:\Users\charl\Documents\GOST\Yemen\graphtool'
    df.to_csv(os.path.join(compath, ofile.replace('.csv', '_snapped.csv')))

    print('Time elapsed: %d seconds' % (time.time() - start))

Beginning snap
Time elapsed: 195 seconds
